# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, concat, count, desc, explode, lit, min, max, split, stddev, udf
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import RegexTokenizer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.clustering import KMeans
from pyspark.sql import Window

import datetime

import os

# Set spark environments
os.environ['PYSPARK_PYTHON'] = '/Users/stillqe/anaconda/envs/Sparkify/bin/python3'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/Users/stillqe/anaconda/envs/Sparkify/bin/python3'



In [5]:
# create a Spark session
spark = SparkSession.builder \
    .master("local") \
    .appName("Sparkify") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [17]:
sample = 'mini_sparkify_event_data.json'
df = spark.read.json(sample)
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [18]:
df.show(1)

+--------------+---------+---------+------+-------------+--------+---------+-----+---------------+------+--------+-------------+---------+---------+------+-------------+--------------------+------+
|        artist|     auth|firstName|gender|itemInSession|lastName|   length|level|       location|method|    page| registration|sessionId|     song|status|           ts|           userAgent|userId|
+--------------+---------+---------+------+-------------+--------+---------+-----+---------------+------+--------+-------------+---------+---------+------+-------------+--------------------+------+
|Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|Bakersfield, CA|   PUT|NextSong|1538173362000|       29|Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|
+--------------+---------+---------+------+-------------+--------+---------+-----+---------------+------+--------+-------------+---------+---------+------+-------------+--------------------+------+
only showi

### Explore what pages we have

In [19]:
df.select('page').dropDuplicates().show(30)

+--------------------+
|                page|
+--------------------+
|              Cancel|
|    Submit Downgrade|
|         Thumbs Down|
|                Home|
|           Downgrade|
|         Roll Advert|
|              Logout|
|       Save Settings|
|Cancellation Conf...|
|               About|
| Submit Registration|
|            Settings|
|               Login|
|            Register|
|     Add to Playlist|
|          Add Friend|
|            NextSong|
|           Thumbs Up|
|                Help|
|             Upgrade|
|               Error|
|      Submit Upgrade|
+--------------------+



In [20]:
df.groupBy('userId').count().orderBy(desc('count')).show()

+------+-----+
|userId|count|
+------+-----+
|    39| 9632|
|      | 8346|
|    92| 7230|
|   140| 6880|
|300011| 5732|
|   124| 4825|
|300021| 4659|
|300017| 4428|
|    85| 4370|
|    42| 4257|
|200023| 3769|
|     6| 3761|
|    29| 3603|
|    54| 3437|
|   100| 3214|
|     9| 3191|
|   126| 3102|
|300015| 3051|
|    91| 3014|
|    98| 2891|
+------+-----+
only showing top 20 rows



In [9]:
# filter for users with blank user id
blank_pages = df.filter(df.userId == '') \
    .select(col('page') \
    .alias('blank_pages')) \
    .dropDuplicates()

# get a list of possible pages that could be visited
all_pages = df.select('page').dropDuplicates()

# find values in all_pages that are not in blank_pages
# these are the pages that the blank user did not go to
for row in set(all_pages.collect()) - set(blank_pages.collect()):
    print(row.page)

Upgrade
Submit Downgrade
Save Settings
Cancel
Downgrade
Thumbs Down
Settings
Roll Advert
Cancellation Confirmation
Submit Upgrade
Logout
Thumbs Up
NextSong
Add to Playlist
Add Friend


In [21]:
df.where(df.page=='NextSong').groupBy('gender').agg({'song':'count'}).show()

+------+-----------+
|gender|count(song)|
+------+-----------+
|     F|     126696|
|     M|     101412|
+------+-----------+



In [22]:
df.filter(df.page == 'NextSong') \
    .select('Artist') \
    .groupBy('Artist') \
    .agg({'Artist':'count'}) \
    .withColumnRenamed('count(Artist)', 'Artistcount') \
    .sort(desc('Artistcount')) \
    .show(5)

+--------------------+-----------+
|              Artist|Artistcount|
+--------------------+-----------+
|       Kings Of Leon|       1841|
|            Coldplay|       1813|
|Florence + The Ma...|       1236|
|       Dwight Yoakam|       1135|
|            BjÃÂ¶rk|       1133|
+--------------------+-----------+
only showing top 5 rows



In [ ]:
df.select('userId').dropDuplicates().count()

In [23]:
churned_usr = df.filter(df.page == 'Cancellation Confirmation').select('userId').dropDuplicates()
churned_usr = churned_usr.withColumn('churn', lit('1'))
df = df.join(churned_usr, 'userId', 'outer')
df = df.fillna('0','churn')
df.select(['userId', 'churn','song']).groupby('churn').agg({'song': 'count'}).show()

+-----+-----------+
|churn|count(song)|
+-----+-----------+
|    0|     191714|
|    1|      36394|
+-----+-----------+



In [24]:
df.select(['userId', 'churn']).dropDuplicates().groupby('churn').count().show()

+-----+-----+
|churn|count|
+-----+-----+
|    0|  174|
|    1|   52|
+-----+-----+



In [25]:
import pyspark.sql.functions as F
def count_missings(spark_df,sort=True):
    """
    Counts number of nulls and nans in each column
    """
    df = spark_df.select([F.count(F.when(F.isnan(c) | F.isnull(c), c)).alias(c) for c in spark_df.columns]).toPandas()

    if len(df) == 0:
        print("There are no any missing values!")
        return None

    if sort:
        return df.rename(index={0: 'count'}).T.sort_values("count",ascending=False)

    return df

count_missings(df)

,count
length,58392
artist,58392
song,58392
location,8346
registration,8346
userAgent,8346
firstName,8346
gender,8346
lastName,8346
ts,0


In [26]:
df = df.dropna(thresh=13)
count_missings(df)

,count
song,50046
length,50046
artist,50046
userId,0
page,0
userAgent,0
ts,0
status,0
sessionId,0
registration,0


In [28]:
df.select('status').dropDuplicates().show(50)

+------+
|status|
+------+
|   307|
|   404|
|   200|
+------+



In [29]:
df.where(df.status=='307').select('page').dropDuplicates().show()

+----------------+
|            page|
+----------------+
|          Cancel|
|Submit Downgrade|
|     Thumbs Down|
|          Logout|
|   Save Settings|
|      Add Friend|
|       Thumbs Up|
|  Submit Upgrade|
+----------------+



In [31]:
get_time = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).strftime('%Y-%m-%d %H:%M:%S'))
df = df.withColumn('time', get_time(df.ts))
df.filter(df.userId == '30').select(['page', 'song', 'registration', 'sessionId', 'ts', 'time']).sort('ts').show()

+---------------+--------------------+-------------+---------+-------------+-------------------+
|           page|                song| registration|sessionId|           ts|               time|
+---------------+--------------------+-------------+---------+-------------+-------------------+
|       NextSong|           Rockpools|1538173362000|       29|1538352117000|2018-09-30 17:01:57|
|       NextSong|   Time For Miracles|1538173362000|       29|1538352394000|2018-09-30 17:06:34|
|       NextSong|Harder Better Fas...|1538173362000|       29|1538352676000|2018-09-30 17:11:16|
|       NextSong|Passengers (Old A...|1538173362000|       29|1538352899000|2018-09-30 17:14:59|
|Add to Playlist|                null|1538173362000|       29|1538352905000|2018-09-30 17:15:05|
|       NextSong|          Fuck Kitty|1538173362000|       29|1538353084000|2018-09-30 17:18:04|
|       NextSong|                Jade|1538173362000|       29|1538353218000|2018-09-30 17:20:18|
|       NextSong|   So-Called 

In [32]:
df.where(df.userId == '30').select(['page', 'sessionId', 'ts', 'time']).show(100)

+---------------+---------+-------------+-------------------+
|           page|sessionId|           ts|               time|
+---------------+---------+-------------+-------------------+
|       NextSong|       29|1538352117000|2018-09-30 17:01:57|
|       NextSong|       29|1538352394000|2018-09-30 17:06:34|
|       NextSong|       29|1538352676000|2018-09-30 17:11:16|
|       NextSong|       29|1538352899000|2018-09-30 17:14:59|
|Add to Playlist|       29|1538352905000|2018-09-30 17:15:05|
|       NextSong|       29|1538353084000|2018-09-30 17:18:04|
|       NextSong|       29|1538353218000|2018-09-30 17:20:18|
|       NextSong|       29|1538353441000|2018-09-30 17:24:01|
|       NextSong|       29|1538353687000|2018-09-30 17:28:07|
|       NextSong|       29|1538353909000|2018-09-30 17:31:49|
|       NextSong|       29|1538354132000|2018-09-30 17:35:32|
|       NextSong|       29|1538354365000|2018-09-30 17:39:25|
|       NextSong|       29|1538354584000|2018-09-30 17:43:04|
|       

In [21]:


cancel = udf(lambda cancled : int(cancled == 'Cancellation Confirmation'), IntegerType())

user_window = Window \
    .partitionBy('userId') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

chu = df \
    .select('userId', 'page', 'ts') \
    .withColumn('churn', cancel(col('page'))).withColumn('churned', Fsum(col('churn')).over(user_window))

52

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.